## Скачать курсы валют

In [260]:
# Запрос у пользователя года
year = input("Введите год (например, 2023): ")

# Запрос у пользователя месяца
month = input("Введите месяц в числовом формате (например, 7 для июля): ")

# Проверка и преобразование введенных данных в числовой формат
try:
    year = int(year)
    month = int(month)
    print(f"Введенный год: {year}, введенный месяц: {month}")
except ValueError:
    print("Некорректный ввод. Пожалуйста, введите числа.")

Введите год (например, 2023): 2023
Введите месяц в числовом формате (например, 7 для июля): 12
Введенный год: 2023, введенный месяц: 12


In [268]:
# Даты начала и окончания периода
start_date = datetime(year, month, 1)
end_date = datetime(year, month, 31)

In [269]:
import requests
from datetime import datetime, timedelta

# Функция для получения курсов валют с сайта ЦБ РФ
def get_cbr_rates(start_date, end_date, currency_codes):
    rates = {code: {} for code in currency_codes}
    base_url = "http://www.cbr.ru/scripts/XML_daily.asp"

    current_date = start_date
    while current_date <= end_date:
        params = {'date_req': current_date.strftime('%d/%m/%Y')}
        response = requests.get(base_url, params=params)
        root = ET.fromstring(response.content)

        for code in currency_codes:
            valute = root.find(f".//Valute[CharCode='{code}']")
            if valute is not None:
                value = float(valute.find('Value').text.replace(',', '.'))
                rates[code][current_date] = value

        current_date += timedelta(days=1)

    return rates

# Получение данных
cbr_data = get_cbr_rates(start_date, end_date, ["USD", "EUR", "HKD", "UZS"])

# Преобразование словаря rates в DataFrame
def format_rates_for_df(rates):
    formatted = []
    for pair, dates in rates.items():
        if isinstance(dates, dict):
            for date, rate in dates.items():
                formatted.append({
                    'Date': date,
                    'CurrencyPair': pair,
                    'Rate': rate
                })
    return formatted

# Форматирование данных от каждого источника
formatted_cbr_data = format_rates_for_df(cbr_data)

# Создание DataFrame для каждого набора данных
df_cbr = pd.DataFrame(formatted_cbr_data)


# Преобразование даты в тип datetime и установка ее как индекса
df_cbr['Date'] = pd.to_datetime(df_cbr['Date'])
df_cbr.set_index('Date', inplace=True)

# Создание словаря для хранения данных по каждой валютной паре
rates_dict = defaultdict(dict)

# Наполнение словаря данными
for index, row in df_cbr.iterrows():
    rates_dict[row['CurrencyPair']][index] = row['Rate']
    
# Преобразование словаря в DataFrame
rates_df_cbr = pd.DataFrame(rates_dict)

# Переименование столбцов
rates_df_cbr.rename(columns={'USD': 'USD/RUB'}, inplace=True)
rates_df_cbr.rename(columns={'EUR': 'EUR/RUB'}, inplace=True)
rates_df_cbr.rename(columns={'HKD': 'HKD/RUB'}, inplace=True)
rates_df_cbr.rename(columns={'UZS': 'RUB/UZS'}, inplace=True)

# Вывод преобразованного DataFrame
print(rates_df_cbr)

            USD/RUB   EUR/RUB  HKD/RUB  RUB/UZS
2023-12-01  88.5819   96.8827  11.3610  72.1350
2023-12-02  89.7619   97.9126  11.5094  73.0960
2023-12-03  89.7619   97.9126  11.5094  73.0960
2023-12-04  89.7619   97.9126  11.5094  73.0960
2023-12-05  90.6728   98.5768  11.6232  73.7172
2023-12-06  91.5823   99.1340  11.7353  74.6270
2023-12-07  92.7826  100.1353  11.9013  75.4981
2023-12-08  92.5654   99.8131  11.8674  75.2248
2023-12-09  91.6402   98.8409  11.7503  74.4729
2023-12-10  91.6402   98.8409  11.7503  74.4729
2023-12-11  91.6402   98.8409  11.7503  74.4729
2023-12-12  90.9846   98.0769  11.6767  73.8992
2023-12-13  90.2158   97.4030  11.5721  73.1855
2023-12-14  89.8926   96.9500  11.5247  72.7934
2023-12-15  89.8926   96.9500  11.5247  72.7934
2023-12-16  89.8926   96.9500  11.5247  72.7934
2023-12-17  89.8926   96.9500  11.5247  72.7934
2023-12-18  89.8926   96.9500  11.5247  72.7934
2023-12-19  89.8926   96.9500  11.5247  72.7934
2023-12-20  89.8926   96.9500  11.5247  

In [270]:
import requests
from datetime import datetime, timedelta

# Функция для получения курса EUR/UZS с сайта Центробанка Узбекистана
def get_cbu_rates(start_date, end_date, currency_code):
    rates = {}
    base_url = "https://cbu.uz/ru/arkhiv-kursov-valyut/xml"

    current_date = start_date
    while current_date <= end_date:
        formatted_date = current_date.strftime('%Y-%m-%d')
        url = f"{base_url}/{currency_code}/{formatted_date}/"
        response = requests.get(url)
        if response.status_code == 200:
            root = ET.fromstring(response.content)
            for ccy in root.findall('CcyNtry'):
                if ccy.find('Ccy').text == currency_code:
                    rate = ccy.find('Rate').text
                    nominal = ccy.find('Nominal').text
                    rates[current_date] = (float(rate), int(nominal))

        current_date += timedelta(days=1)

    return rates

# Получение данных
cbu_data = get_cbu_rates(start_date, end_date, "EUR")

# Преобразование словаря rates в DataFrame
def format_rates_for_df(rates):
    formatted = []
    for pair, dates in rates.items():
        if isinstance(dates, dict):
            for date, rate in dates.items():
                formatted.append({
                    'Date': date,
                    'CurrencyPair': pair,
                    'Rate': rate
                })
        elif isinstance(dates, tuple):
            # Предполагаем, что данные представлены в формате (rate, nominal)
            rate, nominal = dates
            formatted.append({
                'Date': pair,  # В этом случае pair фактически является датой
                'CurrencyPair': 'EUR/UZS',  # Используйте соответствующий код валютной пары
                'Rate': rate,
            })
    return formatted

# Преобразование полученных данных
formatted_data = format_rates_for_df(cbu_data)

# Создание DataFrame
df_cbu = pd.DataFrame(formatted_data)

# Преобразование даты в тип datetime и установка ее как индекса
df_cbu['Date'] = pd.to_datetime(df_cbu['Date'])
df_cbu.set_index('Date', inplace=True)

# Создание словаря для хранения данных по каждой валютной паре
rates_dict = defaultdict(dict)

# Наполнение словаря данными
for index, row in df_cbu.iterrows():
    rates_dict[row['CurrencyPair']][index] = row['Rate']
    
# Преобразование словаря в DataFrame
rates_df_cbu = pd.DataFrame(rates_dict)

# Вывод преобразованного DataFrame
print(rates_df_cbu)

             EUR/UZS
2023-12-01  13415.91
2023-12-02  13415.91
2023-12-03  13415.91
2023-12-04  13404.64
2023-12-05  13345.80
2023-12-06  13311.88
2023-12-07  13278.51
2023-12-08  13278.51
2023-12-09  13278.51
2023-12-10  13278.51
2023-12-11  13272.31
2023-12-12  13272.49
2023-12-13  13329.51
2023-12-14  13340.28
2023-12-15  13340.28
2023-12-16  13340.28
2023-12-17  13340.28
2023-12-18  13340.28
2023-12-19  13340.28
2023-12-20  13340.28
2023-12-21  13340.28
2023-12-22  13340.28
2023-12-23  13340.28
2023-12-24  13340.28
2023-12-25  13340.28
2023-12-26  13340.28
2023-12-27  13340.28
2023-12-28  13340.28
2023-12-29  13340.28
2023-12-30  13340.28
2023-12-31  13340.28


In [271]:
import requests
import xml.etree.ElementTree as ET
from datetime import datetime, timedelta
import pandas as pd
from collections import defaultdict

def fetch_data_for_period(url):
    response = requests.get(url)
    if response.status_code == 200 and response.content.strip():
        try:
            tree = ET.ElementTree(ET.fromstring(response.content))
            return {obs.find('.//generic:ObsDimension', namespaces={'generic': 'http://www.sdmx.org/resources/sdmxml/schemas/v2_1/data/generic'}).attrib['value']: obs.find('.//generic:ObsValue', namespaces={'generic': 'http://www.sdmx.org/resources/sdmxml/schemas/v2_1/data/generic'}).attrib['value'] for obs in tree.getroot().findall('.//generic:Obs', namespaces={'generic': 'http://www.sdmx.org/resources/sdmxml/schemas/v2_1/data/generic'})}
        except ET.ParseError:
            pass
    return {}

def get_ecb_currency_rates(start_date, end_date):
    base_url = "https://data-api.ecb.europa.eu/service/data/EXR"
    series_keys = {
        "EUR.USD": "D.USD.EUR.SP00.A",
        "EUR.CHF": "D.CHF.EUR.SP00.A",
        "EUR.HKD": "D.HKD.EUR.SP00.A"
    }
    filled_data = []

    for pair, series_key in series_keys.items():
        data = fetch_data_for_period(f"{base_url}/{series_key}?startPeriod={start_date.strftime('%Y-%m-%d')}&endPeriod={end_date.strftime('%Y-%m-%d')}")
        last_known_rate = next(iter(data.values()), None)

        for single_date in (start_date + timedelta(days=n) for n in range((end_date - start_date).days + 1)):
            rate = data.get(single_date.strftime('%Y-%m-%d'), last_known_rate)
            last_known_rate = rate if rate else last_known_rate
            if rate:
                filled_data.append({'Date': single_date.strftime('%Y-%m-%d'), 'CurrencyPair': pair.replace('.', '/'), 'Rate': rate})

    return filled_data

currency_rates_data = get_ecb_currency_rates(start_date, end_date)

# Преобразование данных в DataFrame
df = pd.DataFrame(currency_rates_data)

# Преобразование даты в тип datetime и установка ее как индекса
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True)

# Создание словаря для хранения данных по каждой валютной паре
rates_dict = defaultdict(dict)

# Наполнение словаря данными
for index, row in df.iterrows():
    rates_dict[row['CurrencyPair']][index] = row['Rate']
    
# Преобразование словаря в DataFrame
rates_df_ecb = pd.DataFrame(rates_dict)

# Конвертация всех столбцов в числовой формат, удаляя кавычки и преобразуя в float
for col in rates_df_ecb.columns:
    rates_df_ecb[col] = rates_df_ecb[col].astype(str).str.replace('"', '').astype(float)

# Вывод преобразованного DataFrame
print(rates_df_ecb)

            EUR/USD  EUR/CHF  EUR/HKD
2023-12-01   1.0875   0.9530   8.4970
2023-12-02   1.0875   0.9530   8.4970
2023-12-03   1.0875   0.9530   8.4970
2023-12-04   1.0868   0.9476   8.4929
2023-12-05   1.0817   0.9456   8.4562
2023-12-06   1.0778   0.9429   8.4173
2023-12-07   1.0771   0.9446   8.4174
2023-12-08   1.0777   0.9438   8.4190
2023-12-09   1.0777   0.9438   8.4190
2023-12-10   1.0777   0.9438   8.4190
2023-12-11   1.0757   0.9478   8.3971
2023-12-12   1.0804   0.9443   8.4377
2023-12-13   1.0787   0.9452   8.4288
2023-12-14   1.0787   0.9452   8.4288
2023-12-15   1.0787   0.9452   8.4288
2023-12-16   1.0787   0.9452   8.4288
2023-12-17   1.0787   0.9452   8.4288
2023-12-18   1.0787   0.9452   8.4288
2023-12-19   1.0787   0.9452   8.4288
2023-12-20   1.0787   0.9452   8.4288
2023-12-21   1.0787   0.9452   8.4288
2023-12-22   1.0787   0.9452   8.4288
2023-12-23   1.0787   0.9452   8.4288
2023-12-24   1.0787   0.9452   8.4288
2023-12-25   1.0787   0.9452   8.4288
2023-12-26  

In [272]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import pandas as pd
from dateutil.relativedelta import relativedelta

def parse_year_and_month_from_link(link):
    file_name = link.split('/')[-1]  # Например, 'fx_nov23_en.xlsx'
    year_month_part = file_name.split('_')[1]  # Например, 'nov23'
    year = int('20' + year_month_part[-2:])  # Преобразование в год, например 2023
    month_str = year_month_part[:-2]  # Например, 'nov'
    try:
        month = datetime.strptime(month_str, '%b').month  # Преобразование в номер месяца
    except ValueError:
        month = datetime.strptime(month_str, '%B').month  # Если полное название месяца
    return year, month

def get_excel_links_for_period(start_date, end_date):
    url = "https://www.centralbank.ae/en/forex-eibor/exchange-rates/"
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        links = soup.find_all('a')
        filtered_links = []
        # Добавляем предыдущий месяц к периоду
        start_date_with_previous_month = start_date - relativedelta(months=1)
        # Устанавливаем первый день предыдущего месяца
        start_date_with_previous_month = start_date_with_previous_month.replace(day=1)
        for link in links:
            if 'href' in link.attrs and link['href'].endswith('.xlsx'):
                year, month = parse_year_and_month_from_link(link['href'])
                link_date = datetime(year, month, 1)
                if link_date >= start_date_with_previous_month and link_date <= end_date:
                    filtered_links.append("https://www.centralbank.ae" + link['href'])
        return filtered_links
    else:
        print("Ошибка доступа к веб-странице")
        return []

def get_euro_exchange_rates_from_excel(url):
    try:
        df = pd.read_excel(url, header=2).drop(0)
        euro_data = df[df['Currency'] == 'Euro']
        return euro_data
    except Exception as e:
        print(f"Ошибка при загрузке или фильтрации данных из {url}: {e}")
        return pd.DataFrame()

excel_links = get_excel_links_for_period(start_date, end_date)
all_euro_data = pd.DataFrame()

for link in excel_links:
    euro_data = get_euro_exchange_rates_from_excel(link)
    all_euro_data = pd.concat([all_euro_data, euro_data])

# Сортировка данных по дате
all_euro_data.sort_values(by='Date', inplace=True)

# Заполнение пропущенных дат
all_dates = pd.date_range(start=start_date, end=end_date)
all_euro_data.set_index('Date', inplace=True)
all_euro_data = all_euro_data.reindex(all_dates, method='ffill')

# Удаление дат вне заданного диапазона
all_euro_data = all_euro_data.loc[start_date:end_date]

# Создание словаря для хранения данных по каждой валютной паре
rates_dict = defaultdict(dict)

# Наполнение словаря данными
for index, row in all_euro_data.iterrows():
    rates_dict[row['Currency']][index] = row['Rate']
    
# Преобразование словаря в DataFrame
rates_df_aecb = pd.DataFrame(rates_dict)

# Переименование столбца Euro в 'EUR/AED'
rates_df_aecb.rename(columns={'Euro': 'EUR/AED'}, inplace=True)

# Вывод преобразованного DataFrame
print(rates_df_aecb)

             EUR/AED
2023-12-01  4.007092
2023-12-02  4.007092
2023-12-03  4.007092
2023-12-04  4.007092
2023-12-05  4.007092
2023-12-06  4.007092
2023-12-07  4.007092
2023-12-08  4.007092
2023-12-09  4.007092
2023-12-10  4.007092
2023-12-11  4.007092
2023-12-12  4.007092
2023-12-13  4.007092
2023-12-14  4.007092
2023-12-15  4.007092
2023-12-16  4.007092
2023-12-17  4.007092
2023-12-18  4.007092
2023-12-19  4.007092
2023-12-20  4.007092
2023-12-21  4.007092
2023-12-22  4.007092
2023-12-23  4.007092
2023-12-24  4.007092
2023-12-25  4.007092
2023-12-26  4.007092
2023-12-27  4.007092
2023-12-28  4.007092
2023-12-29  4.007092
2023-12-30  4.007092
2023-12-31  4.007092


In [284]:
import pandas as pd
from pathlib import Path
from pandas import ExcelWriter

rates_df = pd.concat([rates_df_ecb, rates_df_aecb, rates_df_cbu, rates_df_cbr], axis=1)

# Предполагается, что rates_df - это ваш объединенный DataFrame

# Получение пути к папке "Загрузки" для текущего пользователя
downloads_path = str(Path.home() / 'Downloads')

# Формирование полного пути к файлу Excel
excel_filepath = downloads_path + '/exchange_rates.xlsx'

# Сохранение DataFrame в Excel
with ExcelWriter(file_path, date_format='DD.MM.YYYY', datetime_format='DD.MM.YYYY') as writer:
    rates_df.to_excel(writer)

print("Excel file has been saved to", excel_filepath)
rates_df

Excel file has been saved to /Users/ilyasilkin/Downloads/exchange_rates.xlsx


<ipython-input-284-2debcdbb8be2>:16: UserWarning: Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.3.7' currently installed).
  with ExcelWriter(file_path, date_format='DD.MM.YYYY', datetime_format='DD.MM.YYYY') as writer:


,EUR/USD,EUR/CHF,EUR/HKD,EUR/AED,EUR/UZS,USD/RUB,EUR/RUB,HKD/RUB,RUB/UZS
2023-12-01,1.0875,0.9530,8.4970,4.007092,13415.91,88.5819,96.8827,11.3610,72.1350
2023-12-02,1.0875,0.9530,8.4970,4.007092,13415.91,89.7619,97.9126,11.5094,73.0960
2023-12-03,1.0875,0.9530,8.4970,4.007092,13415.91,89.7619,97.9126,11.5094,73.0960
2023-12-04,1.0868,0.9476,8.4929,4.007092,13404.64,89.7619,97.9126,11.5094,73.0960
2023-12-05,1.0817,0.9456,8.4562,4.007092,13345.80,90.6728,98.5768,11.6232,73.7172
2023-12-06,1.0778,0.9429,8.4173,4.007092,13311.88,91.5823,99.1340,11.7353,74.6270
2023-12-07,1.0771,0.9446,8.4174,4.007092,13278.51,92.7826,100.1353,11.9013,75.4981
2023-12-08,1.0777,0.9438,8.4190,4.007092,13278.51,92.5654,99.8131,11.8674,75.2248
2023-12-09,1.0777,0.9438,8.4190,4.007092,13278.51,91.6402,98.8409,11.7503,74.4729
2023-12-10,1.0777,0.9438,8.4190,4.007092,13278.51,91.6402,98.8409,11.7503,74.4729
